In [ ]:
@file:DependsOn("/data/repos/actin-personalization/ncr/target/ncr-local-SNAPSHOT-jar-with-dependencies.jar")

import com.hartwig.actin.personalization.datamodel.PatientRecord
import com.hartwig.actin.personalization.ncr.datamodel.NcrRecord
import com.hartwig.actin.personalization.ncr.interpretation.PatientRecordFactory
import com.hartwig.actin.personalization.ncr.serialization.NcrDataReader

val records = NcrDataReader.read("/data/patient_like_me/ncr/K23244.csv")

In [ ]:
val patients = PatientRecordFactory.create(records)

In [ ]:
patients.size

In [ ]:
import com.hartwig.actin.personalization.datamodel.TumorEpisodes
import com.hartwig.actin.personalization.datamodel.TreatmentName
import com.hartwig.actin.personalization.datamodel.SystemicTreatment
import com.hartwig.actin.personalization.datamodel.SystemicTreatmentScheme

fun treatmentSchemesForTumorEpisodes(tumorEpisodes: TumorEpisodes) =
    tumorEpisodes.diagnosisEpisode.systemicTreatmentSchemes!! + tumorEpisodes.followupEpisodes.flatMap { e -> e.systemicTreatmentSchemes!! }

fun schemeToTreatmentNameSet(scheme: SystemicTreatmentScheme) = 
    scheme.treatments.map(SystemicTreatment::treatmentName)
        .map { 
            if (it == TreatmentName.TEGAFUR || it == TreatmentName.TEGAFUR_OR_GIMERACIL_OR_OTERACIL) {
                TreatmentName.FLUOROURACIL
            } else it
        }
        .toSet()

In [ ]:
import com.hartwig.actin.personalization.datamodel.TreatmentName

// Unique treatment schemes by treatment name sets

val treatmentSchemeSets = patients.flatMap { it.episodesPerTumorOfInterest.values }
    .flatMap(::treatmentSchemesForTumorEpisodes)
    .map(::schemeToTreatmentNameSet)
    .groupBy({ it })
    .mapValues { (_, values) -> values.count() }
    .entries.sortedByDescending { (_, count) -> count }
    .takeWhile { (_, count) -> count >= 10 }
    
println(treatmentSchemeSets.map { (names, count) -> "$names: $count" }.joinToString("\n"))
println(treatmentSchemeSets.size)


In [ ]:
import com.hartwig.actin.personalization.datamodel.SystemicTreatment
import com.hartwig.actin.personalization.datamodel.TreatmentName

// Unique patient tumors by treatment name sets in all schemes

val tumorTreatmentSets = patients.flatMap { it.episodesPerTumorOfInterest.values }
    .map { treatmentSchemesForTumorEpisodes(it).map(::schemeToTreatmentNameSet).toSet() }
    .groupBy({ it })
    .mapValues { (_, values) -> values.count() }
    .entries.sortedByDescending { (_, count) -> count }
    //.takeWhile { (_, count) -> count >= 10 }

In [ ]:
println(tumorTreatmentSets.map { (names, count) -> "$names: $count" }.joinToString("\n"))
println(tumorTreatmentSets.size)